### Response Generation

This file will generate the responses for CounselChat questions using 4 models

1. Base GPT (gpt-4o)
2. Fine-Tuned GPT (gpt-4o)
3. Base LLaMA (LLaMA-3.2 3B Instruct)
4. Fine-Tuned LLaMA

In [4]:
import os

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
from unsloth import FastLanguageModel
import torch
from openai import OpenAI
from tqdm import tqdm
import pandas as pd
import pickle
import numpy as np

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


### Read the Processed CounselChat Validation Dataset

In [7]:
with open('processed_data/counselchat_top_votes_test.pkl', 'rb') as file:
    dataset_top_votes_test = pickle.load(file)

dataset_top_votes_test.head()

,topic,question,answerText
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,..."
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi..."
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched..."
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...


Initializing a response generation dataframe to record all the answers

In [8]:
df_response_generation = dataset_top_votes_test
df_response_generation.head()

,topic,question,answerText
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,..."
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi..."
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched..."
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...


### OpenAI Configuration and Responses

In [9]:
with open("../../api.key", 'r') as file:
    openai_api_key = file.read()

openai_client = OpenAI(api_key=openai_api_key)

### OpenAI Base Model

In [10]:
def get_openai_response_base(user_prompt: str) -> str:
        
    completion = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health. Limit your response to a maximum of 215 words"},
        {"role": "user", "content": user_prompt}
        ]
    )

    openai_response = completion.choices[0].message.content
    
    return openai_response

Get GPT Responses

In [11]:
# gpt_responses_base = []
# for index, row in tqdm(dataset_top_votes_test.iterrows(), total=len(dataset_top_votes_test)):
#     question_input = row['question']
#     try:
#         gpt_resp = get_openai_response_base(user_prompt = question_input)
#         gpt_responses_base.append(gpt_resp)
#     except:
#         gpt_responses_base.append('')
    
# with open('response_generation_data/gpt_responses_base.pkl', 'wb') as file:
#     pickle.dump(gpt_responses_base, file)

In [12]:
with open('response_generation_data/gpt_responses_base.pkl', 'rb') as file:
    gpt_responses_base = pickle.load(file)

In [13]:
df_response_generation['gpt_responses_base'] = gpt_responses_base
df_response_generation.head()

,topic,question,answerText,gpt_responses_base
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...,I'm sorry to hear about your experience. It's ...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,...",A healthy and lasting marriage often hinges on...
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi...",It's great that you want to support your frien...
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched...",Navigating intimacy issues in a new marriage c...
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...,I'm sorry to hear you're going through this. I...


### OpenAI Fine-Tuned Model Response

In [14]:
def get_openai_response_finetuned(user_prompt: str) -> str:
        
    completion = openai_client.chat.completions.create(
    model="ft:gpt-4o-2024-08-06:university-of-texas-at-austin:counselchat-train:BGJdvzQV",
    # temperature=0, #setting temperature=0 generates repeated and long responses (avoid in fine-tuned models)
    messages=[
        {"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
        {"role": "user", "content": user_prompt}
        ],
    max_tokens=2048
    )

    openai_response = completion.choices[0].message.content
    
    return openai_response

In [15]:
# gpt_responses_ft = []
# for index, row in tqdm(dataset_top_votes_test.iterrows(), total=len(dataset_top_votes_test)):
#     question_input = row['question']
#     try:
#         gpt_resp = get_openai_response_finetuned(user_prompt=question_input)
#         gpt_responses_ft.append(gpt_resp)
#     except:
#         gpt_responses_ft.append('')
    
# with open('response_generation_data/gpt_responses_ft.pkl', 'wb') as file:
#     pickle.dump(gpt_responses_ft, file)

In [16]:
with open('response_generation_data/gpt_responses_ft.pkl', 'rb') as file:
    gpt_responses_ft = pickle.load(file)

In [17]:
df_response_generation['gpt_responses_ft'] = gpt_responses_ft
df_response_generation.head()

,topic,question,answerText,gpt_responses_base,gpt_responses_ft
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...,I'm sorry to hear about your experience. It's ...,Be straightforward with how you felt at that s...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,...",A healthy and lasting marriage often hinges on...,"What a happy, healthy marriage looks or feels ..."
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi...",It's great that you want to support your frien...,Hi there. It sounds like you care a lot for Ti...
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched...",Navigating intimacy issues in a new marriage c...,You have most of the knowledge you need alread...
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...,I'm sorry to hear you're going through this. I...,There are a number of resources you can utiliz...


### Inferencing from LLaMA Base Model

Preparing the batches of data

In [18]:
batch_size = 10
question_list = dataset_top_votes_test['question'].to_list()
batch_indices = np.arange(0, len(question_list), batch_size)
if batch_indices[-1] != len(question_list):
    batch_indices = np.append(batch_indices, len(question_list))

Loading the base model from Unsloth

In [19]:
max_seq_length = 2048 
dtype = None # None for auto-detection.
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.413 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Implementing batch Inference

In [20]:
def get_llama_response_base(question_inputs: str):
    
    llama_inputs = [[{"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health. Limit your response to a maximum of 200 words"},
                     {"role": "user", "content": question}] for question in question_inputs]

    prompt = tokenizer.apply_chat_template(llama_inputs, tokenize=False, add_generation_prompt=True)
    
    inputs = tokenizer(prompt, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    temp_texts = tokenizer.batch_decode(inputs['input_ids'], skip_special_tokens=True)
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_seq_length,
        num_return_sequences=1
    )

    texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    texts = [i[len(temp_texts[idx]):] for idx, i in enumerate(texts)]
    
    return texts

In [21]:
# # Implementing the Unsloth Fast Inference
# FastLanguageModel.for_inference(model)

# llama_responses_base = []
# for i in tqdm(range(0, len(batch_indices) - 1)):
#     questions_input = question_list[batch_indices[i]:batch_indices[i+1]]
#     llama_resp = get_llama_response_base(questions_input)
#     llama_responses_base = llama_responses_base + llama_resp
    
# with open('response_generation_data/llama_responses_base.pkl', 'wb') as file:
#     pickle.dump(llama_responses_base, file)

In [22]:
with open('response_generation_data/llama_responses_base.pkl', 'rb') as file:
    llama_responses_base = pickle.load(file)

In [23]:
df_response_generation['llama_responses_base'] = llama_responses_base
df_response_generation.head()

,topic,question,answerText,gpt_responses_base,gpt_responses_ft,llama_responses_base
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...,I'm sorry to hear about your experience. It's ...,Be straightforward with how you felt at that s...,It's completely understandable that you're fee...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,...",A healthy and lasting marriage often hinges on...,"What a happy, healthy marriage looks or feels ...",A healthy marriage is built on a foundation of...
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi...",It's great that you want to support your frien...,Hi there. It sounds like you care a lot for Ti...,I'm glad you're taking the initiative to suppo...
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched...",Navigating intimacy issues in a new marriage c...,You have most of the knowledge you need alread...,I can sense the frustration and concern in you...
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...,I'm sorry to hear you're going through this. I...,There are a number of resources you can utiliz...,I'm so sorry to hear that you're struggling wi...


### Inferencing from LLaMA Fine-Tune Model

Preparing the batches of data

In [24]:
batch_size = 10
question_list = dataset_top_votes_test['question'].to_list()
batch_indices = np.arange(0, len(question_list), batch_size)
if batch_indices[-1] != len(question_list):
    batch_indices = np.append(batch_indices, len(question_list))

Loading the fine-tuned model

In [25]:
model_id = "llama32-sft-fine-tune-counselchat"

max_seq_length = 2048 
dtype = None # None for auto-detection.
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.413 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Implementing batch Inference

In [26]:
def get_llama_response_ft(question_inputs: str):
    
    llama_inputs = [[{"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health."},
                     {"role": "user", "content": question}] for question in question_inputs]

    prompt = tokenizer.apply_chat_template(llama_inputs, tokenize=False, add_generation_prompt=True)
    
    inputs = tokenizer(prompt, padding=True, truncation=True, return_tensors="pt")
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    temp_texts = tokenizer.batch_decode(inputs['input_ids'], skip_special_tokens=True)
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_seq_length,
        num_return_sequences=1
    )

    texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    texts = [i[len(temp_texts[idx]):] for idx, i in enumerate(texts)]
    
    return texts

In [27]:
# # Implementing the Unsloth Fast Inference
# FastLanguageModel.for_inference(model)

# llama_responses_ft = []
# for i in tqdm(range(0, len(batch_indices) - 1)):
#     questions_input = question_list[batch_indices[i]:batch_indices[i+1]]
#     llama_resp = get_llama_response_ft(questions_input)
#     llama_responses_ft = llama_responses_ft + llama_resp

# with open('response_generation_data/llama_responses_ft.pkl', 'wb') as file:
#     pickle.dump(llama_responses_ft, file)

In [28]:
with open('response_generation_data/llama_responses_ft.pkl', 'rb') as file:
    llama_responses_ft = pickle.load(file)

In [29]:
df_response_generation['llama_responses_ft'] = llama_responses_ft
df_response_generation.head()

,topic,question,answerText,gpt_responses_base,gpt_responses_ft,llama_responses_base,llama_responses_ft
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...,I'm sorry to hear about your experience. It's ...,Be straightforward with how you felt at that s...,It's completely understandable that you're fee...,It's hard to understand why you're mad. It's p...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,...",A healthy and lasting marriage often hinges on...,"What a happy, healthy marriage looks or feels ...",A healthy marriage is built on a foundation of...,Good question! There are many things that can ...
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi...",It's great that you want to support your frien...,Hi there. It sounds like you care a lot for Ti...,I'm glad you're taking the initiative to suppo...,"Hi. You're asking a lot of questions, and I'll..."
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched...",Navigating intimacy issues in a new marriage c...,You have most of the knowledge you need alread...,I can sense the frustration and concern in you...,I think you're being overly critical of your w...
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...,I'm sorry to hear you're going through this. I...,There are a number of resources you can utiliz...,I'm so sorry to hear that you're struggling wi...,I'm so sorry to hear that you are struggling w...


In [31]:
df_response_generation['llama_responses_ft'][0:50]

0     It's hard to understand why you're mad. It's p...
1     Good question! There are many things that can ...
2     Hi. You're asking a lot of questions, and I'll...
3     I think you're being overly critical of your w...
4     I'm so sorry to hear that you are struggling w...
5     I'm sorry that you are going through this. It'...
6     I'm glad you're reaching out. I think it's a g...
7     There are many reasons why a child would be pl...
8     Hi. I'm so glad you're asking this question. I...
9     I would recommend you learn some relaxation te...
10    erson P P P P P P P P P P P P P P P P P P P P ...
11    ending PPSuggest PSSuggest PSSuggest PSSuggest...
12    erson P P P P P P P P P P P P P P P P P P P P ...
13    ayn P I'm sure you are rightThat you are right...
14    ersonally, I'm a little sad.  I had a bad drea...
15    ersonally You are Pissing Me OffI don't think ...
16    I think the most important thing you can do he...
17    ersonally, I have been a college student f

#### Saving the compiled responses for all the models

In [26]:
with open('response_generation_data/all_model_responses.pkl', 'wb') as file:
    pickle.dump(df_response_generation, file)

In [28]:
with open('response_generation_data/all_model_responses.pkl', 'rb') as file:
    df_response_generation = pickle.load(file)

df_response_generation.head()

,topic,question,answerText,gpt_responses_base,gpt_responses_ft,llama_responses_base,llama_responses_ft
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...,I'm sorry to hear about your experience. It's ...,Be straightforward with how you felt at that s...,It's completely understandable that you're fee...,It's hard to understand why you're mad. It's p...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,...",A healthy and lasting marriage often hinges on...,"What a happy, healthy marriage looks or feels ...",A healthy marriage is built on a foundation of...,Good question! There are many things that can ...
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi...",It's great that you want to support your frien...,Hi there. It sounds like you care a lot for Ti...,I'm glad you're taking the initiative to suppo...,"Hi. You're asking a lot of questions, and I'll..."
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched...",Navigating intimacy issues in a new marriage c...,You have most of the knowledge you need alread...,I can sense the frustration and concern in you...,I think you're being overly critical of your w...
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...,I'm sorry to hear you're going through this. I...,There are a number of resources you can utiliz...,I'm so sorry to hear that you're struggling wi...,I'm so sorry to hear that you are struggling w...


In [1]:
df_response_generation

NameError: name 'df_response_generation' is not defined